# EGM

**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Model](#toc2_)    
- 3. [Numba](#toc3_)    
  - 3.1. [Compile](#toc3_1_)    
  - 3.2. [Euler-error](#toc3_2_)    
  - 3.3. [Transfer func](#toc3_3_)    
  - 3.4. [Multiple Rs and transfer funcs](#toc3_4_)    
  - 3.5. [Save](#toc3_5_)    
  - 3.6. [Time](#toc3_6_)    
- 4. [C++](#toc4_)    
  - 4.1. [Compile](#toc4_1_)    
  - 4.2. [Time](#toc4_2_)    
  - 4.3. [No threading](#toc4_3_)    
  - 4.4. [R](#toc4_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import torch
import time
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})
from egm import retired_income

import matplotlib as mpl
from scipy.stats import binned_statistic

mpl.rcParams.update({
    "font.family": "serif",                  # Make serif the default
    "mathtext.fontset": "cm",                 # Match math font to LaTeX
})

In [ ]:
from BufferStockModel import BufferStockModelClass
from BufferStockModelEGM import BufferStockModelEGMClass
from EconDLSolvers import choose_gpu

## 2. <a id='toc2_'></a>[Simulate Income process](#toc0_)

In [ ]:
model = BufferStockModelEGMClass() 

In [ ]:

model.simulate_income()

In [ ]:
par = model.par
sim = model.sim


# levels
avg_working_income_before_unemp = sim.working_income_before_unemp.mean(axis=1)
avg_working_income_after_unemp = sim.working_income_after_unemp.mean(axis=1)
avg_deterministic_income = sim.deterministic_income.mean(axis=1)
avg_fixed_effect_income_type = sim.fixed_effect_income_type.mean(axis=1)
avg_trend_income_type = sim.trend_income_type.mean(axis=1)
avg_transitory_income = sim.transitory_income.mean(axis=1)
avg_persistent_income = sim.persistent_income.mean(axis=1)
avg_unemployment = sim.unemployment.mean(axis=1)
avg_income = sim.income.mean(axis=1)
std_income = sim.income.std(axis=1)


#  log versions
log_avg_deterministic_income = np.log(sim.deterministic_income).mean(axis=1)
log_avg_fixed_effect_income_type = np.log(sim.fixed_effect_income_type).mean(axis=1)
log_avg_trend_income_type = np.log(sim.trend_income_type).mean(axis=1)
log_transitory_income = np.log(sim.transitory_income).mean(axis=1)
log_persistent_income = np.log(sim.persistent_income).mean(axis=1)
log_income = np.log(sim.income*1000).mean(axis=1)
log_income_var = np.log(sim.income*1000).var(axis=1)
eta_mean = sim.shocks[...,0].mean(axis=1)
epsilon_mean = sim.shocks[...,1].mean(axis=1) 

## 3. <a id='toc2_'></a>[Profiles](#toc0_)

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(25,15))
index_income_work = np.arange(0,par.T_retired)

age_range= np.arange(25,par.T_retired+25)

ax[0,0].plot(age_range, avg_working_income_after_unemp[index_income_work],label="Earnings before taxes and transfers", linewidth = 3 , color = colors[0])
ax[0,0].set_title("Earnings before taxes and transfers - mean", fontsize=25)

ax[0,0].set_xlabel("Age", fontsize=20)
ax[0,0].set_ylabel("$1000", fontsize=20)
# change font size for ticks
ax[0,0].tick_params(axis='both', which='major', labelsize=20)

ax[0,1].plot(age_range, avg_income[index_income_work],label="Disposable income - mean", linewidth = 3 , color = colors[0])
ax[0,1].set_title("Disposable income - mean", fontsize=25)
ax[0,1].set_xlabel("Age", fontsize=20)
ax[0,1].set_ylabel("$1000", fontsize=20)
# change font size for ticks
ax[0,1].tick_params(axis='both', which='major', labelsize=20)

ax[1,0].plot(age_range, log_income[index_income_work],label="Log disposable income", linewidth = 3 , color = colors[0])
ax[1,0].set_title("Log disposable income - mean", fontsize=25)
ax[1,0].set_xlabel("Age", fontsize=20)
ax[1,0].set_ylabel("Log($)", fontsize=20)
# change font size for ticks
ax[1,0].tick_params(axis='both', which='major', labelsize=20)


ax[1,1].plot(age_range, log_income_var[index_income_work],label="Log variance of disposable income", linewidth = 3 , color = colors[0])
ax[1,1].set_title("Log disposable income - variance", fontsize=25)
ax[1,1].set_xlabel("Age", fontsize=20)
ax[1,1].set_ylabel("variance of log($)", fontsize=20)
# change font size for ticks
ax[1,1].tick_params(axis='both', which='major', labelsize=20)

fig.tight_layout()

fig.savefig("income.svg", dpi=300)

In [ ]:
# Step 1: Find unique values and their counts
values, counts = np.unique(model.sim.years_employed, return_counts=True)

# Step 2: Normalize counts to get probabilities
probs = counts / counts.sum()

# Step 3: Cumulative sum to get CDF
cdf = np.cumsum(probs)



fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(values, cdf, marker='o', linestyle='-')

# Set y-axis ticks at 0.0, 0.1, ..., 1.0
ax.set_yticks(np.arange(0.0, 1.01, 0.1))
ax.set_yticklabels([f"{i:.1f}" for i in np.arange(0.0, 1.01, 0.1)])
ax.set_xticks(np.arange(values.min(), values.max() + 1, 5))
ax.set_xticklabels([f"{int(i)}" for i in np.arange(values.min(), values.max() + 1, 5)])

ax.set_xlabel("Years Employed")
ax.set_ylabel("CDF")

fig.savefig("CDF_years_employed.pdf", dpi=300, bbox_inches='tight')

### 3.1 <a id='toc2_'></a>[Decomposition](#toc0_)

In [ ]:
fig,ax = plt.subplots(3,3,figsize=(25,15))
index = np.arange(1,par.T_retired)
index_income = np.arange(0,par.T)

ax[0,0].plot(avg_working_income_before_unemp[index],label="Working income before unemployment")
ax[0,0].set_title("Working income before unemployment")
ax[0,1].plot(avg_working_income_after_unemp[index],label="Working income after unemployment")
ax[0,1].set_title("Working income after unemployment")
ax[0,2].plot(avg_deterministic_income[index],label="Deterministic income")
ax[0,2].set_title("Deterministic income")
ax[1,0].plot(avg_fixed_effect_income_type[index],label="Fixed effect income type")
ax[1,0].set_title("Fixed effect income type")
ax[1,1].plot(avg_trend_income_type[index],label="Trend income type")
ax[1,1].set_title("Trend income type")
ax[1,2].plot(avg_transitory_income[index],label="Transitory income")
ax[1,2].set_title("Transitory income")
ax[2,0].plot(avg_persistent_income[index],label="Persistent income")
ax[2,0].set_title("Persistent income")
ax[2,1].plot(avg_unemployment[index],label="Unemployment")
ax[2,1].set_title("Unemployment")
ax[2,2].plot(avg_income[index_income],label="Income")
# ax[2,2].plot(std_income,label="Income")
ax[2,2].set_title("Income")

fig.tight_layout()


## 4. <a id='toc2_'></a>[Life-time earnings](#toc0_)

In [ ]:
average_earnings = np.mean(model.sim.states[-1,:,2])
print("Average lifetime earnings: ", average_earnings)

### Inequality across specifications

In [ ]:
model.egm.sol_con = None
# Define parameter specifications
specs = {
    'baseline': {},
    'no_het_fixed': {'sigma_alpha': 0.0, 'sigma_theta': 0.0},
    'no_het': {'sigma_alpha': 0.0, 'sigma_theta': 0.0, 'sigma_z1_0': 0.0},
    'no_unemployment': {'a_p_unemp': -50.0, 'b_p_unemp': 0.0, 'c_p_unemp': 0.0, 'd_p_unemp': 0.0},
    'no_persistent': {'disable_z_income': True},
    'no_transitory': {'mu_epsilon_1': 0.0, 'sigma_epsilon_1': 0.0, 'sigma_epsilon_2': 0.0},
}

# Initialize container for results
results = {}

# Loop through specs
for name, override_par in specs.items():
    model = BufferStockModelEGMClass(par =override_par)
    # model.egm.sol_con = None
    model.simulate_income()

    sim = model.sim
    le = sim.lifetime_earnings_compare
    indicator = le > 50
    le_valid = le[indicator]

    log_le = np.log(le_valid)
    std_log = np.std(log_le)
    pct_10, pct_50, pct_90, pct_99 = np.percentile(le_valid, [10, 50, 90, 99])
    results[name] = {
        'Std. dev of log': std_log,
        'P90/P10': pct_90 / pct_10,
        'P90/P50': pct_90 / pct_50,
        'P50/P10': pct_50 / pct_10,
        'P99/P10': pct_99 / pct_10
        }


# Create transposed DataFrame
df_results = pd.DataFrame(results).T.round(4)
print(df_results.T)

In [ ]:
def df_to_latex_tabular(df, caption=None):
    from io import StringIO
    buffer = StringIO()
    buffer.write("\\begin{tabular}{l" + "r" * df.shape[1] + "}\n")
    buffer.write("\\toprule\n")
    buffer.write(" & " + " & ".join(df.columns) + " \\\\\n")
    buffer.write("\\midrule\n")
    for row_label, row in df.iterrows():
        values = " & ".join(f"{v:.2f}" if pd.notnull(v) else "" for v in row)
        buffer.write(f"{row_label} & {values} \\\\\n")
    buffer.write("\\bottomrule\n")
    buffer.write("\\end{tabular}\n")
    return buffer.getvalue()

In [ ]:
latex_str = df_to_latex_tabular(df_results.T)
print(latex_str)

## 5. <a id='toc2_'></a>[Regressing life-time earnings](#toc0_)

In [ ]:

indicator = (sim.states[0,:,3+7] == 1)

fig, ax = plt.subplots(figsize=(10, 6))
# Create a binned scatterplot


n_bins = 100
bin_means, bin_edges, _ = binned_statistic(sim.states[-1,:,1][indicator], sim.states[-1,:,2][indicator], statistic='mean', bins=n_bins)
# bin_means, bin_edges, _ = binned_statistic(sim.states[-1,:,1], sim.states[-1,:,2], statistic='mean', bins=n_bins)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])


ax.scatter(bin_centers, bin_means, marker='o', color=colors[0], label='Mean Lifetime Earnings by z')
ax.set_xlabel('z')
ax.set_ylabel('Mean Lifetime Earnings')

fig.savefig("mean_lifetime_earnings_by_z.pdf", dpi=300, bbox_inches='tight')

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

results_list = []

for i_type in range(model.par.Ntypes):

    sim = model.sim
    indicator = (sim.states[0,:,3+i_type] == 1)

    # Extract final-period states
    annual_le = sim.states[-1, :, 2][indicator]  # Lifetime earnings
    z = sim.states[-1, :, 1][indicator]  # z variable

    # Regression of annual_le on z
    data = pd.DataFrame({'annual_le': annual_le, 'z': z})
    X = data[['z']]  # Use z and z_squared as predictors

    X = sm.add_constant(X)  # add intercept
    y = annual_le
    reg = sm.OLS(y, X).fit()

    # Save results
    results_list.append({
        'type': i_type,
        'intercept': reg.params['const'],
        'z_coef': reg.params[1],  # or reg.params['z'] if named
        'n_obs': reg.nobs
    })

# Combine results into a DataFrame
coef_df = pd.DataFrame(results_list)

print(coef_df)

In [ ]:
latex_table = (
    coef_df[['type', 'intercept', 'z_coef']]  # Include 'type' column
    .rename(columns={
        'type': 'Type',
        'intercept': 'Intercept',
        'z_coef': 'Coefficient',
        'r_squared': '$R^2$'
    })
    .style
    .format({
        'Type': '{:.0f}',       # Format as integer (no decimals)
        'Intercept': '{:.4f}',   # 4 decimal places
        'Coefficient': '{:.4f}',
        '$R^2$': '{:.4f}'
    })
    .hide(axis='index')
    .to_latex(
        hrules=True,
        column_format='lrrrr',  # Added one 'r' for the new column
        caption='Regression Results by Type',
        label='tab:reg_results',
        position='H'
    )
)

print(latex_table)

In [ ]:
coef_df

In [ ]:
# Get lifetime earnings
actual_le = model.sim.states[-1, :, 2]

# compute predicted lifetime earnings using the regression coefficients
predicted_le = np.zeros_like(actual_le)
for i in range(model.sim.N):
    type_index = np.argmax(model.sim.states[0,i,3:])
    reg_constant = coef_df["intercept"][type_index]
    reg_slope = coef_df["z_coef"][type_index]
    predicted_le[i] = reg_constant + reg_slope * model.sim.states[-1,i,1]  # z is the second state variable in the last period

# Calculate retired income based on predicted and actual lifetime earnings
retired_income_predicted = retired_income(par,predicted_le)
retired_income_predicted = par.lambdaa * np.maximum(retired_income_predicted,par.Y_min)**(1-par.tau)
retired_income_ = retired_income(par,actual_le)
retired_income_ = par.lambdaa * np.maximum(retired_income_,par.Y_min)**(1-par.tau)



In [ ]:
# Create dictionary with the metric as keys and lists as values
q_low = 10
summary_data = {
    'Lifetime earnings - Approximate': [np.mean(predicted_le), np.std(predicted_le), np.percentile(predicted_le, q=q_low), np.percentile(predicted_le, q=95)],
    'Lifetime earnings - Baseline': [np.mean(actual_le), np.std(actual_le), np.percentile(actual_le, q=q_low), np.percentile(actual_le, q=95)],
    'Retirement income - Approximate': [np.mean(retired_income_predicted), np.std(retired_income_predicted), np.percentile(retired_income_predicted, q=q_low), np.percentile(retired_income_predicted, q=95)],
    'Retirement income - Baseline': [np.mean(retired_income_), np.std(retired_income_), np.percentile(retired_income_, q=q_low), np.percentile(retired_income_, q=95)],

}

# Create DataFrame and transpose
summary_df = pd.DataFrame(summary_data, index=["Mean", "Standard Deviation", "10th Percentile", "95th Percentile"]).T
summary_df.index.name = ""  # Optional: give the index a name

# Print DataFrame
print(summary_df)

# Export to LaTeX
latex_code = summary_df.to_latex(float_format="%.4f")
print(latex_code)

### Adding a squared term

In [ ]:
results_list = []

for i_type in range(model.par.Ntypes):

    sim = model.sim
    indicator = (sim.states[0,:,3+i_type] == 1)

    # Extract final-period states
    annual_le = sim.states[-1, :, 2][indicator]  # Lifetime earnings
    z = sim.states[-1, :, 1][indicator]  # z variable
    z_squared = z ** 2  # Optional: add squared term if needed

    # Regression of annual_le on z
    data = pd.DataFrame({'annual_le': annual_le, 'z': z, 'z_squared': z_squared})
    X = data[['z', 'z_squared']]  # Use z and z_squared as predictors
    # add squared term if needed

    X = sm.add_constant(X)  # add intercept
    y = annual_le
    reg = sm.OLS(y, X).fit()

    # Save results
    results_list.append({
        'type': i_type,
        'intercept': reg.params['const'],
        'z_coef': reg.params[1],  # or reg.params['z'] if named
        'z_squared_coef': reg.params[2],  # or reg.params['z_squared'] if named
        'r_squared': reg.rsquared,
        'n_obs': reg.nobs
    })

# Combine results into a DataFrame
coef_df = pd.DataFrame(results_list)

print(coef_df)

In [ ]:
latex_table = (
    coef_df[['type', 'intercept', 'z_coef', 'z_squared_coef', 'r_squared']]  # Include 'type' column
    .rename(columns={
        'type': 'Type',
        'intercept': 'Intercept',
        'z_coef': 'Coefficient on linear z',
        'z_squared_coef': 'Coefficient on squared z',
        'r_squared': '$R^2$'
    })
    .style
    .format({
        'Type': '{:.0f}',       # Format as integer (no decimals)
        'Intercept': '{:.4f}',   # 4 decimal places
        'Coefficient on linear z': '{:.4f}',
        'Coefficient on squared z': '{:.4f}',
        '$R^2$': '{:.4f}'
    })
    .hide(axis='index')
    .to_latex(
        hrules=True,
        column_format='lrrrr',  # Added one 'r' for the new column
        caption='Regression Results by Type',
        label='tab:reg_results',
        position='H'
    )
)

print(latex_table)

In [ ]:
coef_df

In [ ]:
actual_le = model.sim.states[-1, :, 2]


predicted_le = np.zeros_like(actual_le)

for i in range(model.sim.N):
    type_index = np.argmax(model.sim.states[0,i,3:])
    reg_constant = coef_df["intercept"][type_index]
    reg_slope = coef_df["z_coef"][type_index]
    reg_squared = coef_df["z_squared_coef"][type_index]
    predicted_le[i] = reg_constant + reg_slope * model.sim.states[-1,i,1] + reg_squared * model.sim.states[-1,i,1]**2  # z is the second state variable in the last period

retired_income_predicted = retired_income(par,predicted_le)
retired_income_predicted = par.lambdaa * np.maximum(retired_income_predicted,par.Y_min)**(1-par.tau)

retired_income_ = retired_income(par,actual_le)
retired_income_ = par.lambdaa * np.maximum(retired_income_,par.Y_min)**(1-par.tau)




In [ ]:
# Create dictionary with the metric as keys and lists as values
q_low = 10
q_high = 95
summary_data = {
    'Lifetime earnings - Approximate': [np.mean(predicted_le), np.std(predicted_le), np.percentile(predicted_le, q=q_low), np.percentile(predicted_le, q=95)],
    'Lifetime earnings - Baseline': [np.mean(actual_le), np.std(actual_le), np.percentile(actual_le, q=q_low), np.percentile(actual_le, q=95)],
    'Retirement income - Approximate': [np.mean(retired_income_predicted), np.std(retired_income_predicted), np.percentile(retired_income_predicted, q=q_low), np.percentile(retired_income_predicted, q=95)],
    'Retirement income - Baseline': [np.mean(retired_income_), np.std(retired_income_), np.percentile(retired_income_, q=q_low), np.percentile(retired_income_, q=95)],

}

# Create DataFrame and transpose
summary_df = pd.DataFrame(summary_data, index=["Mean", "Standard Deviation", "10th Percentile", "95th Percentile"]).T
summary_df.index.name = ""  # Optional: give the index a name

# Print DataFrame
print(summary_df)

# Export to LaTeX
latex_code = summary_df.to_latex(float_format="%.4f")
print(latex_code)